In [1]:
%pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.3 MB/s eta 0:00:00


In [2]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!git clone https://github.com/dauparas/ProteinMPNN
%cd ./ProteinMPNN
!conda create --name mpnn -y

--2024-08-08 01:38:22--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146836934 (140M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 140.03M  77.2MB/s    in 1.8s    

2024-08-08 01:38:24 (77.2 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [146836934/146836934]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHON

In [3]:
%%bash
source activate mpnn
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch -y

Channels:
 - pytorch
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/mpnn

  added / updated specs:
    - cudatoolkit=11.3
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    ca-certificates-2024.7.2   |       h06a4308_0         127 KB
    certifi-2024.7.4           |  py312h06a4308_0         159 KB
    charset-normalizer-3.3.2   |     pyhd3eb1b0_0          44 KB
    cudatoolkit-11.3.1         |       h2bc3f7f_2       549.3 MB
    ffmpeg-4.3                 |       hf484d3e_0         9.9 MB  pytorch
    filelock-3.13.1            |  py312h06a4308_0          24 KB
    freetype-2.12.1            |       h4a9f257_0         626 KB
    gmp-6.2.1                  |       h295c915_3         544 KB
    

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/utilities/metadata_mpnn.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created mpnn metadata dataframe")

Mounted at /content/drive
Existing generation metadata read in.


In [5]:
import os
root_dir = "/content/drive/MyDrive/Generative_Models/unconditional_generation/"
paths = []
for dirpath, dirnames, filenames in os.walk(root_dir):
  for filename in filenames:
      if "generation_metadata" in filename:
          paths.append(os.path.join(dirpath, filename))

import pandas as pd
all_dfs = []
for file_path in paths:
  df = pd.read_csv(file_path)
  df["dir_path"] = "/".join(file_path.split("/")[:-1])
  all_dfs.append(df)
gen_meta = pd.concat(all_dfs, ignore_index=True)

In [6]:
gen_meta = gen_meta[(gen_meta['entity_id'].notnull()) & (gen_meta['task'].str.contains('backbone'))] #& (gen_meta['model'].isin(['framediff']))]
gen_meta['length'] = None
gen_meta.loc[:,"length"] = gen_meta.loc[:,"conditions"].str.extract('(\d+)')[0].astype(int)
gen_meta['input_file_path'] = (gen_meta.dir_path + '/' + gen_meta.output_file_name)
gen_meta = gen_meta.rename(columns={"output_file_name":"input_file_name"})
gen_meta = gen_meta[['model','length','input_file_path','input_file_name', "entity_id"]]
gen_meta = gen_meta.sample(frac=1) #just shuffling the rows
gen_meta = gen_meta.sort_values(by=['length'], ascending=True)

In [7]:
#All foldingdiff outputs > 128 in length are actually just 128 so we'll ignore them
gen_meta = gen_meta.loc[~((gen_meta['length'] > 128) & (gen_meta['model'] == 'foldingdiff'))]

In [8]:
gen_meta.model.unique()
#gen_meta = gen_meta.loc[gen_meta.model != "foldingdiff",:]

array(['genie_swissprot_l_256', 'framediff', 'RFdiffusion_150it',
       'foldingdiff', 'proteinsgm'], dtype=object)

In [9]:
gen_meta.head()

,model,length,input_file_path,input_file_name,entity_id
36079,genie_swissprot_l_256,14,/content/drive/MyDrive/Generative_Models/uncon...,genie_len14_300099f0-aebe-4d85-892d-ce54c3fe4b...,300099f0-aebe-4d85-892d-ce54c3fe4b96
19857,framediff,14,/content/drive/MyDrive/Generative_Models/uncon...,framediff_len14_0d3555e1-9274-40ca-906e-e25bac...,0d3555e1-9274-40ca-906e-e25bacc03ec8
36076,genie_swissprot_l_256,14,/content/drive/MyDrive/Generative_Models/uncon...,genie_len14_bbf2ae29-6cc0-4b51-a42b-501b7c6d03...,bbf2ae29-6cc0-4b51-a42b-501b7c6d0398
119641,RFdiffusion_150it,14,/content/drive/MyDrive/Generative_Models/uncon...,rfdiffusion_len14_5a70c8f7-d78f-41b1-9b3b-91c1...,5a70c8f7-d78f-41b1-9b3b-91c167c5b6eb
36087,genie_swissprot_l_256,14,/content/drive/MyDrive/Generative_Models/uncon...,genie_len14_2a701480-366c-4bbe-ab8e-62eeb6673e...,2a701480-366c-4bbe-ab8e-62eeb6673e23


In [11]:
from Bio import SeqIO
import time
num_designs = 10

for index, row in gen_meta.iterrows():
  length = row["length"]
  pdb_file= row["input_file_path"]
  output_dir = "/content/drive/MyDrive/Generative_Models/utilities/MPNN_backbone/len" + str(length) + "/" + row["model"]
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  output_fasta = output_dir + "/seqs/" +row["input_file_name"].split(".")[0]+".fa"

  if (not all_metadata_df.empty) and (any(all_metadata_df['output_file_path'].str.contains(output_fasta))): continue #file has already been redesigned
  if length ==100: continue # already did these in a batch, will need to rename
  if length > 200: continue
  os.makedirs(output_dir, exist_ok=True)
  print('\n')
  print('\n')
  print(pdb_file)
  print("out to... " + output_fasta)
  print('\n')
  print('\n')

  meta_data_entry = row.copy()[["model","input_file_path","entity_id"]]
  meta_data_entry["gen_model"] = row["model"]
  meta_data_entry["model"] = "ProteinMPNN"
  meta_data_entry["task"] = "Sequence Redesign"
  meta_data_entry["Timestamp"] = str(datetime.now())
  meta_data_entry['gpu'] = 'T4 GPU'
  meta_data_entry['length'] = length
  meta_data_entry['output_file_path'] = output_fasta
  meta_data_entry['num_designs'] = num_designs

  start_time = time.time()
  mpnn_command = f"""
    source activate mpnn
    python ./protein_mpnn_run.py --pdb_path {pdb_file} --out_folder {output_dir} --num_seq_per_target {num_designs} --sampling_temp "0.1" --seed 0 --batch_size 1 --ca_only
    """
  !{mpnn_command}
  end_time = time.time()
  meta_data_entry['wall_time_task'] = str(end_time-start_time) + " Seconds"

  try:
    records = list(SeqIO.parse(output_fasta, "fasta"))
    for pair in [pair.split('=') for pair in records[0].description.split(', ')[1:]]:
          meta_data_entry[pair[0]] = pair[1]
    for i, record in enumerate(records):
      record.id = row["input_file_name"].split(".")[0] + "_design"+str(i)
      record.name = row['input_file_name']

    SeqIO.write(records, output_fasta, "fasta")
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)
  except FileNotFoundError:
    print(f"The file {output_fasta} does not exist.")






In [ ]:

"""
    records = records[1:]
    for record in records:
      record.id = row['model'] + "-" + row['output_file_name']
    with open(merged_fasta_file, "a") as output_handle:
      SeqIO.write(records, output_handle, "fasta")
"""